# Can we build an ensemble model to help us make a profit? Should this model use a running window or is the nature of gold prices static? These are some of the many questions that we will hope to answer.
## Table of Contents

In [20]:
import pandas as pd 
import numpy as np
#Regular expressions for fine-tuned searching
import re

In [16]:
df = pd.read_csv("financial_regression.csv")

Let's take a look at our dataset:

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3904 entries, 0 to 3903
Data columns (total 40 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date                3904 non-null   object 
 1   sp500 open          3719 non-null   float64
 2   sp500 high          3719 non-null   float64
 3   sp500 low           3719 non-null   float64
 4   sp500 volume        3719 non-null   float64
 5   sp500 high-low      3719 non-null   float64
 6   nasdaq open         3719 non-null   float64
 7   nasdaq high         3719 non-null   float64
 8   nasdaq low          3719 non-null   float64
 9   nasdaq volume       3719 non-null   float64
 10  nasdaq high-low     3719 non-null   float64
 11  us_rates_%          176 non-null    float64
 12  CPI                 176 non-null    float64
 13  usd_chf             3694 non-null   float64
 14  eur_usd             3694 non-null   float64
 15  GDP                 57 non-null     float64
 16  silver

In [4]:
df.head(3)

,date,sp500 open,sp500 high,sp500 low,sp500 close,sp500 volume,sp500 high-low,nasdaq open,nasdaq high,nasdaq low,...,palladium high,palladium low,palladium close,palladium volume,palladium high-low,gold open,gold high,gold low,gold close,gold volume
0,2010-01-14,114.49,115.14,114.42,114.93,115646960.0,0.72,46.26,46.52,46.22,...,45.02,43.86,44.84,364528.0,1.16,111.51,112.37,110.79,112.03,18305238.0
1,2010-01-15,114.73,114.84,113.20,113.64,212252769.0,1.64,46.46,46.55,45.65,...,45.76,44.40,45.76,442210.0,1.36,111.35,112.01,110.38,110.86,18000724.0
2,2010-01-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### As you can see, the close price of an index on one day does not equal the open price of the same index on the following day. In order to account for the change in price after trading hours, we will look at the log returns of the close/open prices only. We will choose to use log returns of closing prices, a running standard deviation value of the same day volume, and the percent difference of high/low. This will allow us to keep as many variables as possible. We will however need to determine if we will be overfitting due to high variable count.
#### Let us determine first how much cleaning we will have to do first.

In [17]:
#Drop the open prices
df = df.drop([col for col in df.columns if 'open' in col], axis=1)

In [18]:
df.head(4)

,date,sp500 high,sp500 low,sp500 close,sp500 volume,sp500 high-low,nasdaq high,nasdaq low,nasdaq close,nasdaq volume,...,platinum high-low,palladium high,palladium low,palladium close,palladium volume,palladium high-low,gold high,gold low,gold close,gold volume
0,2010-01-14,115.14,114.42,114.93,115646960.0,0.72,46.52,46.22,46.39,75209000.0,...,2.1300,45.02,43.86,44.84,364528.0,1.16,112.37,110.79,112.03,18305238.0
1,2010-01-15,114.84,113.20,113.64,212252769.0,1.64,46.55,45.65,45.85,126849300.0,...,1.7500,45.76,44.40,45.76,442210.0,1.36,112.01,110.38,110.86,18000724.0
2,2010-01-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-01-19,115.13,113.59,115.06,138671890.0,1.54,46.64,45.95,46.59,84388200.0,...,3.0278,47.08,45.70,46.94,629150.0,1.38,111.75,110.83,111.52,10467927.0


#### The high - low columns do not reflect change relative to stock price. A better alternative is to use percent difference.

In [43]:
#Let's attain the list of unique asssets that have closing price. This will exclude GDP, etc. that does not have low/high.
unique_assets = [re.findall(r"^(.*?)(?=\sclose)", c)[0] for c in df.columns if "close" in c]
print(unique_assets)

['sp500', 'nasdaq', 'silver', 'oil', 'platinum', 'palladium', 'gold']


In [44]:
df_temp = df.copy()

In [46]:
#For each asset that has low/high: create a column that represents the division of high/low for same day:
for a in unique_assets:
    df.loc[:,f"{a} high/low"] = df.loc[:, f"{a} high"]/df.loc[:,f"{a} low"]

In [48]:
df.head(3)

,date,sp500 high,sp500 low,sp500 close,sp500 volume,sp500 high-low,nasdaq high,nasdaq low,nasdaq close,nasdaq volume,...,gold low,gold close,gold volume,sp500 high/low,nasdaq high/low,silver high/low,oil high/low,platinum high/low,palladium high/low,gold high/low
0,2010-01-14,115.14,114.42,114.93,115646960.0,0.72,46.52,46.22,46.39,75209000.0,...,110.79,112.03,18305238.0,1.006293,1.006491,1.013550,1.014403,1.013336,1.026448,1.014261
1,2010-01-15,114.84,113.20,113.64,212252769.0,1.64,46.55,45.65,45.85,126849300.0,...,110.38,110.86,18000724.0,1.014488,1.019715,1.009804,1.019331,1.010972,1.030631,1.014767
2,2010-01-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now we can drop the low/high columns, as that information is already represented in the division columns

In [41]:
re.findall(r"^(.*?)(?=\sclose)", "sp400 close")[0]

'sp400'

In [39]:
strings = ["sp500 close", "ldf close"]

# Regex pattern to match everything except "close"
pattern = r'^(.*?)(?=\sclose)'

# Apply the regex pattern
matches = [re.match(pattern, s).group(1) for s in strings]

print(matches)

['sp500', 'ldf']
